In [1]:
from random import randint, uniform
from util import generate_problem, verify_witness
from matplotlib import pyplot as plt

In [2]:
def random_gene(T, r):
    num_variables = max([max(t['i'], t['j']) for t in T])
    return [0] + [randint(-r, r) for _ in range(num_variables)]

def sweep_line(intervals):
    L = []
    for l, r in intervals:
        L.append( (l, 1) )
        L.append( (r, -1) )
        
    L.sort(key=lambda a: 10*a[0] - a[1])
    
    c = 0
    top = -1
    top_index = 0
    
    for index, (point, increment) in enumerate(L):
        c += increment
        if c > top:
            top = c
            top_index = index
    
    sol = int( (L[top_index][0] + L[top_index+1][0])/2 )
    
    # returns the number of constraints unsatisfied and time point
    return len(intervals) - top, sol

def walk_gene(gene, T, pick_best_gene=True):
    num_variables = max([max(t['i'], t['j']) for t in T])
    
    constraints_per_variable = [list() for i in range(num_variables + 1)]
    # at index i should contain availability intervals (given other variables are fixed.)
    # remark: existence of an interval is local with respect to one constraint
    # remark: so if intersection is empty, then there is no "safe" place to put this variable
    
    for c in T:
        i, j = c['i'], c['j']
        intervals = c['intervals']
        for l, r in intervals:
            # X_j - X_i in (l, r)
            # fix X_j to genes[j]
            # so genes[j] - X_i in (l, r)
            # then X_i in [X_j - r, X_j - l]
            constraints_per_variable[i].append( (gene[j] - r, gene[j] - l) )
            
            # now for j:
            constraints_per_variable[j].append( (gene[i] + l, gene[i] + r) )
            
            # ^ tested this on paper, it's ok.
    
    
    # for each constraint, sweep line, find best option
    # (later? we want to pick whatever REDUCES the unsatisfieds the most)
    # for now: pick a random variable and modify it according to sweep_line
    # if no later, then we can make the loop upstairs simpler (to only do stuff if "i" is mentioned)

    i = randint(1, num_variables)
    gene[i] = sweep_line(constraints_per_variable[i])[1]

    return gene
    
def walk_tcsp(T, r, max_iterations, max_flips):
    best_gene = None
    best_gene_failed = None
    
    for i in range(max_iterations):
        gene = random_gene(T, r)
        
        for j in range(max_flips):
            gene_failed = verify_witness(gene, T)
            
            if not best_gene or len(gene_failed) < len(best_gene_failed):
                best_gene = gene
                best_gene_failed = gene_failed
            
            if len(gene_failed) == 0:
                break
                
            gene = walk_gene(gene, T)
                
        # necessary for double break after flips loop
        if len(gene_failed) == 0: break
    
    print(f'num iterations: {i+1}')
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [3]:
from pprint import pprint
T = generate_problem(
    20,
    min_intervals=1,
    max_intervals=1,
    scaling_factor=1
)

walk_tcsp(T, 100, 50, 50)

num iterations: 50
best gene: [0, 91, -12, -38, -36, -20, -31, 6, 49, -61, 40, -41, 4, 13, -98, -72, 19, -12, 49, 13]
constraints failed: 7 out of: 58
failed constraints: [{'i': 1, 'j': 3, 'intervals': [(-92, 30)]}, {'i': 1, 'j': 6, 'intervals': [(34, 90)]}, {'i': 2, 'j': 14, 'intervals': [(52, 67)]}, {'i': 2, 'j': 17, 'intervals': [(-88, -74)]}, {'i': 10, 'j': 14, 'intervals': [(-45, 16)]}, {'i': 14, 'j': 15, 'intervals': [(-89, -82)]}, {'i': 14, 'j': 17, 'intervals': [(84, 94)]}]


In [4]:
l = [(0, 10), (1, 5), (1, 6), (6, 10)]

In [5]:
sweep_line(l)

(1, 3)

## Graphing effectiveness of walking a gene

In [9]:
def test(T, r, max_flips):
    unsats = []
    gene = random_gene(T, r)

    for j in range(max_flips):
        gene_failed = verify_witness(gene, T)
        unsats += [len(gene_failed)]

        if len(gene_failed) == 0:
            break

        gene = walk_gene(gene, T)

    return unsats

In [71]:

# problem sets:
SIMPLE = {
    'variables': 20,
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 1,
    'scaling_factor': 1,
}

BINARY = {
    'variables': 20,
    'constraint_probability': 0.25,
    'min_intervals': 2,
    'max_intervals': 2,
    'scaling_factor': 1,
}

FIVE_MAX = {
    'variables': 20,
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 5,
    'scaling_factor': 1,
}


runs = 50
run_results = []

for _ in range(runs):
    T = generate_problem(
        **BINARY
    )
    
    arr = test(T, r=100, max_flips=100)
    
    starting = arr[0]
    ending = arr[-1]
    decrease = (arr[0]-arr[-1]) / arr[0]
    
    # TODO, problem: we do not know if it is even solvable!
    # will have to confirm with backtracking, or make sure to generate only solvable problems
    solved = arr[-1] == 0
    
    print(starting, ending, f'{decrease:.2f}', solved)

20 2 0.90 False
28 6 0.79 False
27 3 0.89 False
22 4 0.82 False
33 4 0.88 False
33 6 0.82 False
40 9 0.78 False
32 7 0.78 False
23 7 0.70 False
25 3 0.88 False
26 4 0.85 False
26 7 0.73 False
38 5 0.87 False
29 6 0.79 False
26 4 0.85 False
27 1 0.96 False
30 5 0.83 False
24 3 0.88 False
34 7 0.79 False
31 5 0.84 False
33 8 0.76 False
20 2 0.90 False
33 8 0.76 False
36 4 0.89 False
33 4 0.88 False
24 6 0.75 False
28 0 1.00 True
34 2 0.94 False
39 7 0.82 False
33 10 0.70 False
27 3 0.89 False
27 6 0.78 False
19 0 1.00 True
36 6 0.83 False
25 4 0.84 False
27 4 0.85 False
24 5 0.79 False
23 2 0.91 False
30 6 0.80 False
23 4 0.83 False
27 5 0.81 False
24 4 0.83 False
22 3 0.86 False
28 5 0.82 False
23 0 1.00 True
27 1 0.96 False
26 5 0.81 False
32 4 0.88 False
31 3 0.90 False
21 2 0.90 False


In [18]:
# Note:
# if returned length is less, that means the heuristic solved the problem
# check whether it ever increases.